# Проект матричное кодирование

### Аннотация

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv('/datasets/insurance.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [2]:
df.sample(20)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4438,1,36.0,48700.0,1,0
3875,1,32.0,65900.0,0,0
3436,0,47.0,29600.0,1,1
3984,1,37.0,29800.0,0,0
4392,0,36.0,35400.0,1,0
4857,1,36.0,43000.0,4,0
3024,0,31.0,36600.0,2,0
4667,0,27.0,42500.0,2,0
1849,1,18.0,39200.0,2,0
4000,1,19.0,43000.0,0,0


In [3]:
df.columns = ['sex','age','income','family_size', 'payment']

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** не изменится,


**Обоснование:**

**Рассмотрим пошаговые преобразования, опираясь на свойства матричных операций:**

- Домножим матрицу X на квадратную матрицу Z (Z - обратимая матрица), тогда заменяя X в исходном уравнении обучения получим:

$$
w = ((XZ)^T XZ)^{-1} (XZ)^T y
$$

- Отсюда уравнения предсказаний принимают такой вид:

$$
a = XZ (((XZ)^T XZ)^{-1} (XZ)^T y)
$$

- Свойство произведения транспонированных матриц: $(AB)^T = B^T A^T $, значит:

$$
a = XZ ((Z^T X^T XZ)^{-1} Z^T X^T y)
$$

- Свойство произведения обратных матриц $(AB)^{-1} = B^{-1}A^{-1}$, тогда:

$$
a = XZZ^{-1}(X^TX)^{-1}(Z^T)^{-1}Z^TX^Ty
$$


- По свойству произведения обратных матриц: $A^{-1}A = AA^{-1} = E $ (единичная матрица), значит:

$$
a = XE(X^TX)^{-1}EX^Ty
$$

- Произведение матрицы на единичную матрицу E: $AE = EA = A$, соответственно:

$$
a = X(X^T)^{-1}X^T y
$$

- Исходная формула - $w = (X^T X)^{-1} X^T y$, значит мы получили исходную формулу предсказаний:
$$
a = Xw
$$

## 3. Алгоритм преобразования

**Алгоритм**


1) Делим исходный датасет фичеры и таргеты

2) Создаем обратную квадратную матрицу размеру 4x4 (поскольку ширина фрейма признаков - 4), которая будет являться ключом кодирования

3) Умножаем на данный ключ матрицу с признаками, оставляя таргет прежним

**Обоснование**

Таким образом, мы сохраняем целостность эксперимента у нас будут варьировать w & w0, чтобы получить один и тот же таргет.

А декодировать преобразованную матрицу мы сможем с помощью умножения на обратную ей.

Таким образом, мы на практике реализуем обоснование данное выше

## 4. Проверка алгоритма

In [4]:
features, target = df.iloc[:,0:4], df.iloc[:,4]
features.head()

,sex,age,income,family_size
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


## Тестирование с помощью преобразований

**Преобразуем матрицу признаков с помощью умножения на случайную квадратную матрицу 4х4:**

In [4]:
A = np.random.randint(1,1e4,(4,4))



print(np.linalg.det(A))

158584312639355.6


**Определитель матрицы не равен нулю - матрица обратима**

In [7]:
A_1 = np.linalg.inv(A)

print(A.dot(A_1))

[[ 1.00000000e+00  1.22016112e-15 -5.07569247e-16 -1.88759598e-16]
 [ 7.95479134e-16  1.00000000e+00  3.68439003e-16  1.25279561e-16]
 [ 1.83772268e-15 -3.39030019e-15  1.00000000e+00  1.09829680e-16]
 [ 2.35217661e-16 -2.03558958e-16 -3.20517267e-17  1.00000000e+00]]


**Создаем новое пространство признаков:**

In [27]:
features_t = features.values @ A

In [28]:
new_features = pd.DataFrame(features_t, columns = features.columns)

**Делим на train и test выборки**

In [29]:
X_train, X_test, y_train, y_test = train_test_split(new_features, target, test_size = .25, random_state = 42)

In [30]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Получаем предсказания и сохраняем значение метрики R2**

In [33]:
predictions_1 = model.predict(X_test)

r2_coded = r2_score(y_test, predictions_1)

In [ ]:
# ОБЫЧНО ТАРГЕТ НА ПЕРВОМ МЕСТЕ СТАВЯТ, ПОТОМ ПРЕДИКТ
r2_score(y_test, predictions_1)

## Тестирование без преобразований

Проиндексируем исходные фичеры и таргеты для чистоты эксперимента

In [34]:
X_train_clear = features.iloc[X_train.index]

X_test_clear = features.iloc[X_test.index]

y_train_clear = target.iloc[y_train.index]
y_test_clear = target.iloc[y_test.index]

In [35]:
model_test = LinearRegression()

model_test.fit(X_train_clear, y_train_clear)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

**Получим предсказания модели для незакодированных фичей и посчитаем R2-score**

In [36]:
predictions_2 = model_test.predict(X_test_clear)

r2_not = r2_score(y_test_clear, predictions_2)

### Сравним значения метрик с одной и той же базой для обучения

In [37]:
print(f'Метрика R2 для модели с преобразованными данными: {r2_coded:.3f}\n\
Метрика R2 для модели с преобразованными данными: {r2_not:.3f}')

Метрика R2 для модели с преобразованными данными: 0.425
Метрика R2 для модели с преобразованными данными: 0.425


## Выводы

**С помощью свойств матричных преобразований, мы преобразовали наши исходные данные, таким образом, что мы метрика качества обученной модели не изменилась существенным образом.**

***Данный способ хоть и является простым, однако он уже позволяет существенным образом скрыть реальную информацию так, чтобы финальный результат не пострадал качеством.***